Dieses Tutorial zeigt die ersten Schritte mit LangChain. Wir werden die notwendigen Bibliotheken installieren und einen ersten Prompt an GPT3.5 (ChatGPT) schicken.

In [2]:
%pip install langchain openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

llm = ChatOpenAI(model="gpt-3.5-turbo")

response = llm.predict("Spreche ich mit ChatGPT?")

print(response)

Ja, Sie sprechen mit ChatGPT. Wie kann ich Ihnen helfen?
